# Advanced VQE: Quantum Subspace Expansion for LiH

## Direct Hamiltonian description

The function ``get_active_space_spin_hamiltonian`` is able to provide only the Hamiltonian. It is very similar to ``ucc``'s ``uccsd`` regarding its parameters.

The first step is to instantiate OpenFermion's ``MolecularData`` class with the chosen basis set and the studied geometry.

Then this object is given as an argument to the function which outputs:
- the Hamiltonian,
- the number of qubits.

**Example:** lithium hydride in 6-31G basis at 1.75 Angström (internuclear distance.)

In [1]:
# Package import:
import numpy as np
from qat.dqs.qchem.deprecated.tools import get_active_space_spin_hamiltonian
from qat.dqs.qchem.deprecated.to_be_converted import perform_ci_and_ed
from qat.dqs.qchem.deprecated.converters import from_QubitOperator_to_IsingHamiltonian
from openfermion.hamiltonians import MolecularData

In [2]:
# Molecule construction:
basis = '6-31g'
multiplicity = 1 # Supposed to be set to 1 (i.e. singlet state is assumed!)
bond_length = 1.75 # In Angstrom.
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., bond_length))]

molecule = MolecularData(geometry, basis, multiplicity)

In [3]:
hamiltonian, nb_qubits = get_active_space_spin_hamiltonian(molecule)

# Optional parameters are:
#  - The transformation: transformation='bk',
#  - The numerical threshold: threshold=1e-15, 
#  - The active space selection criterion: as_selection=0.001,
#        - either the lower limit value for the NOONs of active orbitals, 
#        - or the list of active orbitals.  
#  - The transformation as a BinaryCode, for user-defined ones: code=None, 
#  - The reduction of trivially acted-upon qubits (valid for Braavyi-Kitaev, in certain basis): reduction=False, 
#  - The display of the Hamiltonian under fermionic form: print_fermionic_hamiltonian=False.

/home/tayral/.local/lib/python3.6/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:
/home/tayral/.local/lib/python3.6/site-packages/pyscf/lib/misc.py:876: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  h5py.File.__init__(self, filename, *args, **kwargs)


## Reference value calculation

The quality of the VQE output can be evaluted through comparison with:
- the exact energy i.e. from *Full Configuration Interaction* ;
- the encoded energy i.e. from classical diagonalization of the Hamiltonian matrix.

The second comparison is more relevant as the VQE is first and foremost an eigensolver and is real input is the Hamiltonian (not the "brute" molecule).

The function ``get_reference_values`` always provides the FCI energy (thanks to ``PySCF``) and if the Hamiltonian is of reasonable size, its smallest eigenvalue.

*Note: the Hamiltonian matrix is built using ``scipy.sparse``'s sparse matrices and diagonalized in this framework.*

**Example:** water molecule in 3-21G basis at internuclear distance 1 Angström and bond angle 50°.

In [4]:
# Molecule construction:
basis = '3-21g'
multiplicity = 1 # Supposed to be set to 1 (i.e. singlet state is assumed!)
bond_length = 1 # In Angstrom.
bond_angle = 50 # In degrees.
geometry = [('O', (0., 0., 0.)), 
            ('H', (bond_length, 0., 0.)), 
            ('H', (bond_length*np.cos(bond_angle), bond_length*np.sin(bond_angle), 0.))]

molecule = MolecularData(geometry, basis, multiplicity)

In [5]:
h_ising, nb_qubits = get_active_space_spin_hamiltonian(molecule, 
                                                       as_selection=1e-2) # To reduce the size of the active 
                                                                          # (and thus of the Hamiltonian.)

e_fci, e_diag = perform_ci_and_ed(molecule, h_ising, nb_qubits)
print("FCI and ED energies: ", e_fci, e_diag)

FCI and ED energies:  -74.7825725478948 -74.66228226227454


## Quantum Subspace Expansion

*Source material: https://arxiv.org/abs/1603.05681, https://arxiv.org/abs/1707.06408, https://arxiv.org/abs/1807.10050*

The VQE algorithm exhibits a "natural" robustness against errors, especially regarding $\vec{\theta}^\star$, the optimal value of the parameter. Unfortunately, the energy evalutation (i.e. mean-value measurement) can still suffer from important errors. 

McClean *et al.* drew inspiration from the classical *Linear Response Theory* which is widely used in perturbation theory and error mitigation, to design an efficient extension to the VQE, the *Quantum Subspace Expansion* (QSE). The core idea is to expand the Hamiltonian post-VQE on a well-chosen subspace (i.e. where an improved, lower, energy lies) and solve classically the associated generalized eigeinvalue problem with the hope of getting an improved value for the ground state energy.

More precisely, the QSE can be splitted into different steps:
1. Choice of qubit operators;
2. Expansion of the Hamiltonian on the subspace defined by the two previous choices; Construction of the overlap matrix;
3. Resolution of the generalized eigenvalue problem.

Thus, the $n$-qubit QSE using $G$ as the chosen set of $n$-qubit operators, is associated  with the following state subspace:
$$
    \{ \hat{\sigma}|\psi^\star\rangle, \qquad \hat{\sigma} \in G \}
$$
where $|\psi^\star\rangle = |\mathrm{UCC}(\vec{\theta}^\star)\rangle$ is the output of the VQE.  
The expanded Hamiltonian and overlap matrices, $(H_{i, j})$ and $(S_{i, j})$, are then measured *via* a quantum computer, i.e.
$$
    H_{i, j} = \langle \psi^\star | \hat{\sigma}_i^\dagger \hat{H} \hat{\sigma}_j | \psi^\star\rangle \qquad
    S_{i, j} = \langle \psi^\star | \hat{\sigma}_i^\dagger \hat{\sigma}_j | \psi^\star\rangle
$$
Finally, the associated generalized eigenvalue problem is solved classically and the minimal solution is extracted, i.e.
$$
    E_{\mathrm{QSE}} = \min\{E, \qquad H \vec{x} = E S \vec{x}\}
$$

If $G$ is a set of 1-qubit operators (more exactly, $n$-qubit operators acting non-identically only on 1 qubit), the expansion is said to be *linear*.  
If $G$ is a set of Pauli operators, it is named a Pauli expansion.

*Note: The selection of $G$ is really difficult as some choices can give outwardly bad results. Some of the best results are given by choosing $G$ as the set of symmetry operators of the Hamiltonian.*

**Example:** dihydrogen in STO-3G basis at internuclear distance 0.7414 Angström and simulated on an IBM chip.

In [1]:
from openfermion import MolecularData
from openfermion.ops import QubitOperator
from qat.dqs.qchem.deprecated.ucc import uccsd
from qat.dqs.qchem.deprecated.qse import apply_quantum_subspace_expansion
from qat.dqs.hamiltonian import IsingHamiltonian
from qat.dqs.vqe import VQE

from qat.dqs.optimization import Optimizer
from qat.dqs.spsa import spsa_minimize
spsa_optimizer = Optimizer(spsa_minimize)

In [2]:
# Molecule construction:
basis = 'sto-3g'
multiplicity = 1 # Supposed to be set to 1 (i.e. singlet state is assumed!)
bond_length = 0.7414 # In Angstrom.
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

molecule = MolecularData(geometry, basis, multiplicity)

In [7]:
# Noisy QPU construction:
from qat.quops.quantum_channels import ParametricPureDephasing, ParametricAmplitudeDamping
from qat.hardwares.default import HardwareModel
from qat.hardwares.default import DefaultGatesSpecification

from qat.linalg import get_qpu_server
from qat.noisy import get_qpu_server as get_noisy_qpu_server

def get_ibm_noisy_gpu():

    gate_durations = {"H":50, "X":50,"RY": lambda angle : 200, "RX": lambda angle : 200,"RZ": lambda angle : 200,"CNOT":200, "PH": lambda angle : 200}

    ibm_gates_spec = DefaultGatesSpecification(gate_durations)

    T1, T2 = 40000, 40000 #nanosecs

    amp_damping = ParametricAmplitudeDamping(T_1 = T1)
    pure_dephasing = ParametricPureDephasing(T_phi = 1/(1/T2 - 1/(2*T1)))

    return get_noisy_qpu_server(hardware_model = HardwareModel(ibm_gates_spec,
                                                               idle_noise = [amp_damping, pure_dephasing]),
                                sim_method = "deterministic")
    """
    return get_noisy_qpu_server()
    """

my_noisy_qpu = get_ibm_noisy_gpu()

In [11]:
# State preparation:
qroutine_uccsd, theta_0, h_spin, nb_qubits = uccsd(molecule, transformation='bk', reduction = False)


In [12]:
# VQE execution:
print("theta0=", theta_0)
h_vqe = IsingHamiltonian(list_pauli_operators=h_spin.list_pauli_operators,
                      list_pauli_values = h_spin.list_pauli_values)
e_vqe, param_vqe, nb_evals, energies = VQE(h_vqe, spsa_optimizer, qroutine_uccsd, theta_0, 
                                           my_noisy_qpu, grouping=False, display=False)

print("E VQE=", e_vqe)

theta0= [0.03632537-0.j]
Precision reached ( 0.0001 ), iteration number = 9
E VQE= (-0.968867709591744+0j)


In [11]:
# QSE execution:

#expansion_operators = [QubitOperator(''), QubitOperator('Z0 Z1')]
expansion_operators = [IsingHamiltonian([["I", "I"]], [1.]), IsingHamiltonian([["Z", "Z"]], [1.])]

# Symmetry based choice (see third source material above.)

e_qse = apply_quantum_subspace_expansion(h_spin, 
                                         nb_qubits, 
                                         qroutine_uccsd(param_vqe),
                                         expansion_operators, 
                                         my_noisy_qpu,
                                         return_matrices=False)

In [12]:
# Reference value computation:
e_fci, e_diag = perform_ci_and_ed(molecule, h_spin, nb_qubits)

In [13]:
# Comparison:
print('Energy comparison (gap to encoded energy) in Hartree:')
print('Noisy VQE ground state energy:     {} ({})'.format(e_vqe.real, e_vqe.real - e_diag))
print('Noisy VQE+QSE ground state energy: {} ({})'.format(e_qse, e_qse - e_diag)) 
print('Encoded ground state energy:       {} (0)'.format(e_diag)) 
print('FCI ground state energy:           {}'.format(e_fci)) 

Energy comparison (gap to encoded energy) in Hartree:
Noisy VQE ground state energy:     -1.1372701746002212 (6.068234803535688e-11)
Noisy VQE+QSE ground state energy: -1.137270174600222 (6.068145985693718e-11)
Encoded ground state energy:       -1.1372701746609035 (0)
FCI ground state energy:           -1.1372701746609022
